**Task 07: Querying RDF(s)**

In [1]:
!pip install rdflib
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2024-2025/master/Assignment4/course_materials"


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


First let's read the RDF file

In [5]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
from rdflib.plugins.sparql import prepareQuery
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/rdf/example6.rdf", format="xml")

<Graph identifier=N831dc51fd63f428d9241a4bf35bdd95d (<class 'rdflib.graph.Graph'>)>

**TASK 7.1: List all subclasses of "LivingThing" with RDFLib and SPARQL**

In [8]:
ns = Namespace("http://somewhere#")
task71 = prepareQuery(
  '''
  SELECT ?Subject WHERE {
    ?Subject rdfs:subClassOf* ns:LivingThing.
  }
  ''',
    initNs={'rdfs':RDFS, "ns":ns}
)
# Visualize the results

for r in g.query(task71):
  print(r)

(rdflib.term.URIRef('http://somewhere#LivingThing'),)
(rdflib.term.URIRef('http://somewhere#Person'),)
(rdflib.term.URIRef('http://somewhere#Researcher'),)
(rdflib.term.URIRef('http://somewhere#PhDstudent'),)
(rdflib.term.URIRef('http://somewhere#Professor'),)
(rdflib.term.URIRef('http://somewhere#Animal'),)


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [11]:
task72 = prepareQuery(
  '''
  SELECT ?Subject WHERE {
    ?Subject rdf:type ?class .
    ?class rdfs:subClassOf* ns:Person .
  }
  ''',
    initNs={'rdfs':RDFS,'ns':ns}
)

# Visualize the results
for r in g.query(task72):
  print(r)

(rdflib.term.URIRef('http://somewhere#SaraJones'),)
(rdflib.term.URIRef('http://somewhere#JohnSmith'),)
(rdflib.term.URIRef('http://somewhere#JaneSmith'),)
(rdflib.term.URIRef('http://somewhere#JimGonzalez'),)


**TASK 7.3: List all individuals of just "Person" or "Animal". You do not need to list the individuals of the subclasses of person (in SPARQL only)**


In [13]:
task73 = prepareQuery(
  '''
  SELECT ?Subject WHERE {
    {?Subject rdf:type ns:Person} UNION
    {?Subject rdf:type ns:Animal}
  }
  ''',
    initNs={'rdfs':RDFS, "ns":ns}
)

# Visualize the results
for r in g.query(task73):
  print(r)

(rdflib.term.URIRef('http://somewhere#SaraJones'),)
(rdflib.term.URIRef('http://somewhere#JohnSmith'),)
(rdflib.term.URIRef('http://somewhere#RockySmith'),)
(rdflib.term.URIRef('http://somewhere#Bugs'),)


**TASK 7.4:  List the name of the persons who know Rocky (in SPARQL only)**

In [14]:
from rdflib import FOAF
VCARD = Namespace("http://www.w3.org/2001/vcard-rdf/3.0/")

task74 = prepareQuery(
  '''
  SELECT ?name WHERE {
    ?Subject foaf:knows ns:RockySmith .
    ?Subject vcard:FN ?name .
  }
  ''',
    initNs={"ns":ns, "foaf":FOAF, "vcard" : VCARD}
)

# Visualize the results
for r in g.query(task74):
    print(r.name)

Sara Jones
Jane Smith


**Task 7.5: List the name of those animals who know at least another animal in the graph (in SPARQL only)**

In [15]:
VCARD = Namespace("http://www.w3.org/2001/vcard-rdf/3.0/")
task75 = prepareQuery(
  '''
  SELECT ?name WHERE {
    ?Subject a ns:Animal .
    ?Subject foaf:knows ?Subject2 .
    ?Subject2 a ns:Animal .
    ?Subject vcard:Given ?name
  }
  ''',
    initNs={'rdfs':RDFS, "ns":ns, 'foaf': FOAF,'vcard':VCARD}
)

# Visualize the results
for r in g.query(task75):
    print(r.name)

Rocky
Bugs


**Task 7.6: List the age of all living things in descending order (in SPARQL only)**

In [16]:
task76 = prepareQuery(
  '''
  SELECT ?age WHERE {
    ?class rdfs:subClassOf* ns:LivingThing .
    ?alive rdf:type ?class .
    ?alive foaf:age ?age
  }
  ORDER BY DESC(?age)
  ''',
  initNs={"ns": ns, "vcard": VCARD, "foaf": FOAF}
)

# Visualize the results
for r in g.query(task76):
  print(r.age)

40
30
22
20
10
2
